In [16]:
from set_sum.chaojiying import *
from lxml import etree

In [21]:
#测试爬虫自动识别验证码并登录再爬取信息. 为了爬取需要登录才显示的数据！！！



headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
}

params={
    
}

proxies={
    'http':'113.140.188.66:4256',
    'http':'222.90.41.210:4275'
}

#看看情况，先检测cookie,找cookie的请求地址也不好找啊~~
s = requests.Session()
first_url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx' 
s.get(first_url,headers=headers)

#获取好cookie后，再发送请求
url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'
response = requests.get(url=url,headers=headers)

#解析需要的数据
tree = etree.HTML(response.text)
img_n = 'https://so.gushiwen.cn/' + tree.xpath('//*[@id="imgCode"]/@src')[0] #获取图片请求地址

view_state = tree.xpath('//*[@id="__VIEWSTATE"]/@value')[0]                  #动态参数1
view_generator = tree.xpath('//*[@id="__VIEWSTATEGENERATOR"]/@value')[0]     #动态参数2

#保存图片
code_data = s.get(url=img_n,headers=headers).content
with open('./img_data.jpg','wb') as f:
    f.write(code_data)

#调用程序来解析图片
chaojiying = Chaojiying_Client('注册的用户名', '注册的密码', '')#用户中心>>软件ID 生成一个替换 96001
im = open('img_data.jpg', 'rb').read()                           #本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
img_num = chaojiying.PostPic(im, 1902)['pic_str'] #1902是验证码的类型 1902 验证码类型
print(img_num)

#模拟登录，其中的__Viewstate类似的是动态请求参数,可以先看看页面中搜索下，看是否是隐藏的
data={
    '__VIEWSTATE': view_state,
    '__VIEWSTATEGENERATOR': view_generator,
    'from': 'http://so.gushiwen.cn/user/collect.aspx',
    'email': '古诗网注册的用户名',
    'pwd': '古诗网注册的密码',
    'code': img_num,
    'denglu': '登录'
}

login_url = 'https://so.gushiwen.cn/user/login.aspx?from=http%3a%2f%2fso.gushiwen.cn%2fuser%2fcollect.aspx'
main_page = s.post(url=login_url,headers=headers,data=data)

with open('古诗文用户主页.html','w',encoding='utf-8') as f:
    f.write(main_page.text)

vizn
